In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('C:/Users/Kate/Downloads/Product_Analysis_data2.csv')

In [2]:
df.columns

Index(['user_id', 'event_timestamp', 'first_purchase_time', 'cohort_week',
       'lifetime_weeks', 'product_id'],
      dtype='object')

In [3]:
# Convert time column to datetime type 
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
df['first_purchase_time'] = pd.to_datetime(df['first_purchase_time'])

In [4]:
# turn string values into floats 
df = df.drop(['product_id'], axis=1)
df['purchase'] = 2.49
df

,user_id,event_timestamp,first_purchase_time,cohort_week,lifetime_weeks,purchase
0,6f8592c0-fb4c-47dc-9344-424eb5d88a4b,2023-10-13 09:54:02,2023-10-13 09:54:02,2023-10-08,0,2.49
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-10-14 04:45:14,2023-03-25 04:44:40,2023-03-19,29,2.49
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 04:44:40,2023-03-25 04:44:40,2023-03-19,0,2.49
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-01 04:44:16,2023-03-25 04:44:40,2023-03-19,1,2.49
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-08 04:45:55,2023-03-25 04:44:40,2023-03-19,2,2.49
...,...,...,...,...,...,...
6081,a4c941de-b4a2-431e-b882-38b78f6e6fbb,2023-09-21 06:54:50,2023-09-14 06:54:01,2023-09-10,1,2.49
6082,a4c941de-b4a2-431e-b882-38b78f6e6fbb,2023-09-28 06:54:39,2023-09-14 06:54:01,2023-09-10,2,2.49
6083,a4c941de-b4a2-431e-b882-38b78f6e6fbb,2023-10-05 06:54:19,2023-09-14 06:54:01,2023-09-10,3,2.49
6084,a4c941de-b4a2-431e-b882-38b78f6e6fbb,2023-10-12 06:53:58,2023-09-14 06:54:01,2023-09-10,4,2.49


In [5]:
df['purchase'].sum() # загальний прибуток

15154.140000000005

In [6]:
df['user_id'].nunique()

934

In [7]:
df['purchase'].value_counts() # загальна кількість підписок

2.49    6086
Name: purchase, dtype: int64

In [8]:
df['lifetime_weeks'].mean() #середня тривалість життя клієнта

7.603187643772593

In [9]:
# Calculate revenue for each cohort
cohorts = df.groupby('cohort_week')
revenue_per_cohort = cohorts['purchase'].sum().reset_index()
revenue_per_cohort.rename(columns={'purchase': 'rev_per_cohort'}, inplace=True)
revenue_per_cohort.head() 

,cohort_week,rev_per_cohort
0,2023-02-26,408.36
1,2023-03-05,1591.11
2,2023-03-12,1217.61
3,2023-03-19,1411.83
4,2023-03-26,933.75


In [10]:
revenue_per_cohort.mean()

<ipython-input-10-492e0674f36c>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  revenue_per_cohort.mean()


rev_per_cohort    473.566875
dtype: float64

In [11]:
purchase_for_week.mean()

NameError: name 'purchase_for_week' is not defined

In [ ]:
data = pd.merge(df, revenue_per_cohort, on='cohort_week', how='left')
data

In [ ]:
# CLV / ACL (Customer Lifetime Value * Average Customer Lifetime)
data['LTV'] = data['rev_per_cohort'].mean() / data['lifetime_weeks'].mean() # У ФОРМУЛІ МАЄ БУТИ МНОЖЕННЯ?
data

In [ ]:
# ARPU X CL (Average revenue per user (MRR/total users)  / Customer Lifetime)
data['ARPU'] = 16.22
data['LTV2'] = data['ARPU'] * data['lifetime_weeks']
data

In [ ]:
data['LTV2'].mean()


## Predict LTV using extrapolation

In [ ]:
# Calculate average LTV for each cohort
average_ltv_per_cohort = data.groupby('cohort_week')['LTV'].mean()

# Extrapolate LTV for the year
projected_ltv_year = average_ltv_per_cohort.mean() * 52  # Assuming 52 weeks in a year

print("Projected LTV for the year :", projected_ltv_year)

In [ ]:
# Calculate average LTV for each cohort
average_ltv_per_cohort = data.groupby('cohort_week')['LTV2'].mean()

# Extrapolate LTV for the year
projected_ltv_year = average_ltv_per_cohort.mean() * 52  # Assuming 52 weeks in a year

#print(average_ltv_per_cohort)
print("Projected LTV for the year :", projected_ltv_year)

## Predict LTV using Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


features = ['lifetime_weeks']
X = data[features].values.reshape(-1, len(features))
y = data['LTV2'].values.reshape(-1, len(features))  

In [ ]:
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X, y)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
accuracy = model.score(X_test, y_test)

print("Mean Squared Error:", mse)
print("R-Squared Accuracy:", accuracy)
print("Predicted LTV for the next year: ", '\n', y_pred)

In [ ]:
y_pred = np.round(y_pred, decimals=2)
y_pred = y_pred.flatten().tolist()
dff = pd.DataFrame(y_pred, columns=['LTV_pred'])
print(dff)

In [ ]:
# Create and fit the linear regression model
model_full = LinearRegression()
model_full.fit(X, y)

In [ ]:
full_pred = model_full.predict(X)
accuracy2 = model_full.score(X, y)

print("Predictions", '\n', full_pred)
print("Accuracy", accuracy2)

In [ ]:
full_pred = np.round(full_pred, decimals=2)
full_pred = full_pred.flatten().tolist()
data['LTV2_pred'] = pd.DataFrame(full_pred, columns=['LTV2_pred'])
data

In [ ]:
diff = data['LTV2_pred'] - data['LTV2']
diff.mean()

In [ ]:
data.to_csv (r'C:/Users/Kate/Downloads/LTV_dataframe2.csv', index = None, header=True)